In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [5]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index 
        self.t = t  
        self.true_branch = true_branch  
        self.false_branch = false_branch  

In [6]:
def gain(left_labels, right_labels, root_criterion, criterion):


    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [7]:
def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
def predict_object(obj, node):

   
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return predict_object(obj, node.true_branch)
    else:
        return predict_object(obj, node.false_branch)

In [9]:
def predict(data, tree):
    
    preds = []
    for obj in data:
        prediction = predict_object(obj, tree)
        preds.append(prediction)
    return preds

In [10]:
def print_tree(node, spacing=""):


    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return


    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))


    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

1. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [11]:
data, targets = make_regression(n_features=2, n_informative=2, random_state=5)

In [25]:
data, targets = make_regression(n_features=2, n_informative=2, random_state=5)

In [26]:
class Leaf:
    
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        self.prediction = self.predict()
        
    def predict(self):
        return self.targets.mean()

In [27]:
def mse(targets):
    return np.mean((targets - targets.mean())**2)

In [28]:
def find_best_split(data, targets):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    root_mse = mse(targets)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_targets, false_targets = split(data, targets, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_targets, false_targets, root_mse, mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [29]:
def build_tree(data, target):

    gain, t, index = find_best_split(data, target)

   
    if gain == 0:
        return Leaf(data, target)

    true_data, false_data, true_target, false_target = split(data, target, index, t)

    
    true_branch = build_tree(true_data, true_target)

    false_branch = build_tree(false_data, false_target)

    node = Node(index, t, true_branch, false_branch)

   
    return node

In [32]:
my_tree = build_tree(train_data_regr, train_target_regr)
print_tree(my_tree)

NameError: name 'train_data_regr' is not defined

In [33]:
from sklearn.metrics import r2_score

train_answers = predict(train_data_regr, my_tree)
train_r2 = r2_score(train_target_regr, train_answers)
print(train_r2)

answers = predict(test_data_regr, my_tree)
test_r2 = r2_score(test_target_regr, answers)
print(test_r2)

NameError: name 'train_data_regr' is not defined